In [139]:
import folium, json, requests, glob, os, imageio
from folium import plugins
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull
import selenium.webdriver
from datetime import datetime

In [3]:
def make_base_map(custom_map):
    custom_map = folium.Map(location=[52.510008, 13.404954],
                    zoom_start=12)
    return custom_map

In [4]:
def add_flexzone(custom_map):
    with open(r'C:\Users\steff\Documents\DataScience Bootcamp\Bike\geo\flexzone_bn.json') as f:
        bln_zone = json.load(f)
    # add the polgon of the berlin flexzone
    folium.GeoJson(
            bln_zone,
            name='geojson'
            ).add_to(custom_map)
    return custom_map

In [5]:
bikemap = add_flexzone(make_base_map('bikemap'))

In [11]:
df = pd.read_pickle('all_trips.pkl')
df['date_hour'] = df['trip_start_time'].dt.round('h') 
df.head()

,index,trip_id,bike_id,trip_duration,trip_start_time,trip_end_time,from_station,from_station_id,from_station_mode,from_lat,from_long,to_station,to_station_id,to_station_mode,to_lat,to_long,date_hour
11,998695,20190311011,13022,23,2019-03-11 16:39:41,2019-03-11 17:02:49,Karl-Marx Allee,1465,station,52.5173,13.4413,Oranienplatz,1445,station,52.502,13.4162,2019-03-11 17:00:00
12,998707,20190311012,13022,13,2019-03-11 17:13:53,2019-03-11 17:26:57,Oranienplatz,1445,station,52.502,13.4162,Maybachufer,1589,station,52.4896,13.4391,2019-03-11 17:00:00
14,922715,20190311014,13023,75,2019-03-11 08:47:44,2019-03-11 10:03:18,Helsingforser Platz,1458,station,52.5078,13.4496,BIKE 13023,0,floating,52.4862,13.3765,2019-03-11 09:00:00
18,451393,20190311018,13026,11,2019-03-11 23:06:53,2019-03-11 23:17:57,BIKE 13026,0,floating,52.5473,13.4094,U Senefelder Platz,1543,station,52.5329,13.4124,2019-03-11 23:00:00
20,888466,20190311020,13028,29,2019-03-11 09:04:53,2019-03-11 09:34:05,Sonnenallee/Innstraße,1609,station,52.4806,13.4428,BIKE 13028,0,floating,52.5015,13.4111,2019-03-11 09:00:00


In [63]:
date_hour = sorted(df['date_hour'].unique())

In [ ]:
path = (r"C:\Users\steff\Documents\DataScience Bootcamp\Bike\maps_html")
for dh in date_hour: 
    dt = pd.to_datetime(dh)
    st_h = (str(dt.date())+'_hh'+str(dt.hour))
    bikemap = add_flexzone(make_base_map('bikemap'))
    for index, row in df.iterrows():
        if dt == row['date_hour']:
                    folium.PolyLine(locations=
                                    ([row['from_lat'] ,row['from_long']],
                                     [row['to_lat'] ,  row['to_long']]),
                                opacity = 0.5,
                                color='blue',
                                weight= 5                        
                        ).add_to(bikemap)
                    bikemap.save(path+f'\map{st_h}.html')
        else:
            continue

### making an  gif

In [162]:
# get all html files
path = (r"C:\Users\steff\Documents\DataScience Bootcamp\Bike\maps_html")
allFiles = glob.glob(os.path.join(path,"*.html"))
type(allFiles)

list

In [178]:
# convert html to png
driver = selenium.webdriver.PhantomJS()
for f in allFiles:
    driver.set_window_size(800, 600)  # choose a resolution
    driver.get(r'maps_html/'+f[61:])
    #print()
    # You may need to add time.sleep(seconds) here
    driver.save_screenshot(f'maps_png/map{f[61:-5]}.png')

In [182]:
pic = []
for f in allFiles:
    filename = f.format(f)
    images.append(imageio.imread(f'myplot_{i}.png'))
    pic.append(imageio.imread(f)).format(f)
imageio.mimsave('nextbike_hours.gif', images, fps=20)

NameError: name 'images' is not defined

In [188]:
png_dir = (r"C:\Users\steff\Documents\DataScience Bootcamp\Bike\maps_png\\")
images = []
for file_name in os.listdir(png_dir):
    if file_name.endswith('.png'):
        file_path = os.path.join(png_dir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave("nextbike.gif", images)